In [1]:
import os
import pandas as pd
import zipfile
import csv
from io import StringIO
import mysql.connector
import re

### Testing on home server

In [ ]:
try:
    connection = mysql.connector.connect(
        host = '192.168.1.26',
        user = 'aportra',
        password = '@$Occer22',
        database = 'wedge_assignment')

    cursor = connection.cursor()
except mysql.connector.Error as err:
    print(f'{err}')

finally:
    if connection.is_connected():
        connection.close()
        print('Connection Closed')
    
    if cursor:
        print('Cursor created')

#### Setting Directory for Mac and Windows 

In [19]:
try:
    directory_small = '/Users/aaronportra/Documents/ADA/wedge_project/WedgeZipOfZips_small/'
    directory = '/Users/aaronportra/Documents/ADA/wedge_project/WedgeZipOfZips/'
    os.listdir(directory)
except FileNotFoundError:
    directory_small = 'C:\\Users\\aport\\OneDrive\\Documents\\School\\Fall Semester 2024\\Applied Data Analytics\\wedge_project\\WedgeZipOfZips_small\\'
    directory = 'C:\\Users\\aport\\OneDrive\\Documents\\School\\Fall Semester 2024\\Applied Data Analytics\\wedge_project\\WedgeZipOfZips\\'
    os.listdir(directory)

## Cleaning Data for BQ

In [5]:
def sniff(file):

    file = StringIO(file)

    sample = file.read(10000)

    sniffer = csv.Sniffer()

    file.seek(0)

    delimiter = sniffer.sniff(sample).delimiter


    reader = csv.reader(file, delimiter=delimiter)

    first_row = next(reader)

    has_header = sniffer.has_header(sample)


    if len(first_row) < 50:
        has_header = False

    return delimiter, has_header

In [ ]:
# transactions = {}

# for file in os.listdir(directory):
#     if file.endswith('.zip'):
#         with zipfile.ZipFile(os.path.join(directory, file),'r') as zip_file:
#             for info in zip_file.namelist():
    
#                 file_content = zip_file.read(info).decode('utf-8',errors = 'replace')

#                 csv_file = StringIO(file_content)

#                 h = sniff(file_content)

#                 csv_file.seek(0)

#                 if h[1]:
#                     data = pd.read_csv(csv_file, delimiter=h[0])
#                     data = pd.DataFrame(data)

#                     columns = data.columns


#                 elif not h[1]:
#                     data = pd.read_csv(csv_file, delimiter = h[0], header = None)
#                     data = pd.DataFrame(data)
#                     data.columns = columns
 

#             transactions[file.rstrip('.zip')] = data   

In [10]:
#Removing Null values and replacing with white space
def del_nan(data):
    for trans in data:
        data[trans] = data[trans].fillna('')

#Adding space to text. Example: ChickenBreast -> Chicken Breast
def add_space(text):
    if isinstance(text,str):
        return re.sub(r'([a-z])([A-Z])',r'\1 \2',text)

    return text


In [29]:
def create_db(directory):
    transactions = {}

    for file in os.listdir(directory):
        if file.endswith('.zip'):
            with zipfile.ZipFile(os.path.join(directory, file),'r') as zip_file:
                for info in zip_file.namelist():
        
                    file_content = zip_file.read(info).decode('utf-8',errors = 'replace')

                    csv_file = StringIO(file_content)

                    h = sniff(file_content)

                    csv_file.seek(0)

                    if h[1]:
                        data = pd.read_csv(csv_file, delimiter=h[0])
                        data = pd.DataFrame(data)

                        columns = data.columns


                    elif not h[1]:
                        data = pd.read_csv(csv_file, delimiter = h[0], header = None)
                        data = pd.DataFrame(data)
                        data.columns = columns
    

                transactions[file.rstrip('.zip')] = data

    del_nan(transactions)

    for trans in transactions:
        for column in transactions[trans]:
            if transactions[trans][column].dtypes == 'O':
                transactions[trans][column] = transactions[trans][column].apply(add_space)

    return transactions

In [27]:
small_zips = create_db(directory_small)

# full_zips = create_db(directory)

In [28]:
for i in small_zips:
    for y in small_zips[i]:
        print(small_zips[i][y])

0       2013-10-01 07:11:53
1       2013-10-01 07:11:53
2       2013-10-01 07:11:53
3       2013-10-01 07:11:53
4       2013-10-01 07:11:53
               ...         
9995    2013-10-01 13:58:30
9996    2013-10-01 13:58:33
9997    2013-10-01 13:58:36
9998    2013-10-01 13:58:37
9999    2013-10-01 13:58:42
Name: datetime, Length: 10000, dtype: object
0       700
1       700
2       700
3       700
4       700
       ... 
9995      5
9996      5
9997      4
9998      5
9999      5
Name: register_no, Length: 10000, dtype: int64
0       700
1       700
2       700
3       700
4       700
       ... 
9995     76
9996     76
9997     71
9998     76
9999     76
Name: emp_no, Length: 10000, dtype: int64
0       829
1       829
2       829
3       829
4       829
       ... 
9995     37
9996     37
9997     28
9998     37
9999     37
Name: trans_no, Length: 10000, dtype: int64
0       0000000009118
1       0000000009120
2       0000000009122
3       0000000009123
4       0020398500000
        

In [43]:
save_path = '/Users/aaronportra/Documents/Advanced Data Analytics/wedge_project/clean_transactions/'

for trans in transactions:
    df = transactions[trans].to_csv(save_path + trans + '.csv', encoding = 'utf-8', index = False,)